In [1]:
from pydantic import BaseModel

# Define your Pydantic model to structure the output
class ExpandedAnswer(BaseModel):
    initial_answer: str
    detailed_answer: str


In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define the prompt template
initial_prompt = PromptTemplate.from_template(
    template="Given the context: '{context}', what is the answer to the question: '{question}'?"
)

# Initialize the LLM
# llm = ChatOpenAI(model="gpt-4", temperature=0.7)
llm = ChatOpenAI(
                model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
                base_url="http://10.2.125.37:1234/v1",
                api_key="lm-studio"
            )


In [3]:
# Chain using the pipe syntax and structured output with Pydantic
chain = initial_prompt | llm.with_structured_output(ExpandedAnswer)

In [4]:
# Define the input data
context = "AI is increasingly being adopted in various industries, including healthcare..."
question = "What are the key benefits of AI in healthcare?"

# Run the chain with the input data
response = chain.invoke({
    "context": context,
    "question": question
})

# Access the structured output
print(response.initial_answer)
print(response.detailed_answer)

AttributeError: 'NoneType' object has no attribute 'initial_answer'

In [16]:
from langchain import LLMChain, PromptTemplate
from langchain.llms.base import LLM
from typing import List, Optional
from pydantic import BaseModel, Field
import requests

class LMStudioLLM(LLM):
    base_url: str
    model_name: str
    api_key: Optional[str] = None

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> ExpandedAnswer:
        response = requests.post(
            f"{self.base_url}/v1/chat/completions",
            headers={"Content-Type": "application/json"},
            json={
                "model": self.model_name,
                "messages": [
                    {"role": "system", "content": "Always answer in rhymes."},
                    {"role": "user", "content": prompt}
                ],
                "temperature": 0.7,
                "max_tokens": 150,
                "stream": False
            }
        )
        response_json = response.json()
        content = response_json['choices'][0]['message']['content']
        
        # Assuming the response format contains a summary and details separated by a delimiter
        parts = content.split("\n\n")
        if len(parts) == 2:
            return ExpandedAnswer(initial_answer=parts[0], detailed_answer=parts[1])
        else:
            return ExpandedAnswer(initial_answer=content, detailed_answer="")

    @property
    def _llm_type(self) -> str:
        return "lmstudio"



In [17]:
# Instantiate your custom LLM
llm = LMStudioLLM(base_url="http://10.2.125.37:1234", model_name="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF")

# Define the input data
context = "AI is increasingly being adopted in various industries, including healthcare..."
question = "What are the key benefits of AI in healthcare?"

# Combine context and question into a single prompt
prompt_text = f"Given the context: '{context}', what is the quick summary? Now, please provide a detailed explanation for: '{question}'."

# Get the structured response directly
response = llm._call(prompt_text)

# Access the structured output
print("Initial Answer:", response.initial_answer)
print("Detailed Answer:", response.detailed_answer)



Initial Answer: The adoption's on the rise, I must imply,
In various fields, including healthcare, it does apply.
Detailed Answer: A quick summary: AI's being used to aid,
Improving patient care, and making diagnoses not delayed.
From diagnosis to treatment, AI is playing a part,
Enhancing lives with precision, and a gentle art.


In [19]:
from langchain.prompts import PromptTemplate

content_template="""# CONTEXT #
I want to advertise for my company's duty free product catalogue. The SKU Name of the product is {sku_name}, brand of the product is {brand_name} and its category is : {sub_category}.

# OBJECTIVE #
Create new product description for the above product based on the following existing description: {description}. Extract all key points and features from the existing description and write a new description from them. The word count should be similar to the original description.

# STYLE #
The writing style needs focus on the unique aspects of the product - its ingredients, its process, its history. Use descriptive language to evoke sensory experience - taste, smell, touch. Be precise and concise when writing. The content needs to be simple enough for the common man to understand it.
Following words are BLACKLISTED from appearing in the response: 'transformative','tapestry','like',';','-'.
REPLACE the following words with a more descriptive and specific response: 'foster','fostering','all about','is about','think of','like','but also'.

# TONE #
Simple, clear and elegant.

# AUDIENCE #
My company's audience profile is the common man who likes to indulge in the finer things in life every once in a while.
"""
content_prompt=PromptTemplate(input_variables=["sku_name","brand_name","sub_category","description"],template=content_template)

In [20]:
llm = ChatOpenAI(
                model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
                base_url="http://10.2.125.37:1234/v1",
                api_key="lm-studio"
            )

In [21]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

class Product(BaseModel):
    product_description : str = Field(description="The generated product description")

parser= PydanticOutputParser(pydantic_object=Product)

content_template="""# CONTEXT #
I want to advertise for my company's duty free product catalogue. The SKU Name of the product is {sku_name}, brand of the product is {brand_name} and its category is : {sub_category}.

# OBJECTIVE #
Create new product description for the above product based on the following existing description: {description}. Extract all key points and features from the existing description and write a new description from them. The word count should be similar to the original description.

# STYLE #
The writing style needs focus on the unique aspects of the product - its ingredients, its process, its history. Use descriptive language to evoke sensory experience - taste, smell, touch. Be precise and concise when writing. The content needs to be simple enough for the common man to understand it.
Following words are BLACKLISTED from appearing in the response: 'transformative','tapestry','like',';','-'.
REPLACE the following words with a more descriptive and specific response: 'foster','fostering','all about','is about','think of','like','but also'.

# TONE #
Simple, clear and elegant.

# AUDIENCE #
My company's audience profile is the common man who likes to indulge in the finer things in life every once in a while.

# RESPONSE #
{format_instructions}
"""

content_prompt=PromptTemplate(input_variables=["sku_name","brand_name","sub_category","description"],template=content_template,partial_variables={"format_instructions":parser.get_format_instructions()})

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import OutputFixingParser

llm = ChatOpenAI(
                model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
                base_url="http://10.2.125.37:1234/v1",
                api_key="lm-studio"
            )

class Product(BaseModel):
    product_description : str = Field(description="The generated product description")

content_template="""# CONTEXT #
I want to advertise for my company's duty free product catalogue. The SKU Name of the product is {sku_name}, brand of the product is {brand_name} and its category is : {sub_category}.

# OBJECTIVE #
Create new product description for the above product based on the following existing description: {description}. Extract all key points and features from the existing description and write a new description from them. The word count should be similar to the original description.

# STYLE #
The writing style needs focus on the unique aspects of the product - its ingredients, its process, its history. Use descriptive language to evoke sensory experience - taste, smell, touch. Be precise and concise when writing. The content needs to be simple enough for the common man to understand it.
Following words are BLACKLISTED from appearing in the response: 'transformative','tapestry','like',';','-'.
REPLACE the following words with a more descriptive and specific response: 'foster','fostering','all about','is about','think of','like','but also'.

# TONE #
Simple, clear and elegant.

# AUDIENCE #
My company's audience profile is the common man who likes to indulge in the finer things in life every once in a while.

{format_instructions}
"""

parser= PydanticOutputParser(pydantic_object=Product)
output_fixing_parser=OutputFixingParser(parser=parser,llm=llm)
content_prompt=PromptTemplate(input_variables=["sku_name","brand_name","sub_category","description"],template=content_template,partial_variables={"format_instructions":parser.get_format_instructions()})
content_runnable = content_prompt | llm  | output_fixing_parser



import pprint
user_input={
"brand_name":"Chloe",
"sku_name":"CHLOÉ Atelier des Fleurs Magnolia Alba Eau de Parfum 150ml",
"sub_category":"Perfumes",
"description":"The House of Chloé unveils a collection of nine exclusive Eau de Parfum fragrances for woman: Atelier des Fleurs.\nMagnolia Alba transcribes the smooth, plump and slightly lemony notes of magnolia blossoms in spring."
}

result=content_runnable.invoke(user_input)
pprint.pprint(result.product_description)

("Chloé's Atelier des Fleurs Magnolia Alba Eau de Parfum 150ml is a unique and "
 'exclusive fragrance that captures the essence of spring. The scent features '
 'smooth, plump notes of magnolia blossoms, complemented by subtle lemony '
 'undertones. This exquisite perfume is crafted with precision and care to '
 'evoke a sensory experience that is both elegant and refined.')


In [28]:
content_prompt

PromptTemplate(input_variables=['brand_name', 'description', 'sku_name', 'sub_category'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"product_description": {"title": "Product Description", "description": "The generated product description", "type": "string"}}, "required": ["product_description"]}\n```'}, template="# CONTEXT #\nI want to advertise for my company's duty free product catalogue. The SKU Name of the product is {sku_name}, brand of the product is {brand_name} and its category is : {sub_category}.\n\n# OBJECTIV

In [27]:
content_runnable

PromptTemplate(input_variables=['brand_name', 'description', 'sku_name', 'sub_category'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"product_description": {"title": "Product Description", "description": "The generated product description", "type": "string"}}, "required": ["product_description"]}\n```'}, template="# CONTEXT #\nI want to advertise for my company's duty free product catalogue. The SKU Name of the product is {sku_name}, brand of the product is {brand_name} and its category is : {sub_category}.\n\n# OBJECTIV

In [26]:
import pprint
user_input={
"brand_name":"Chloe",
"sku_name":"CHLOÉ Atelier des Fleurs Magnolia Alba Eau de Parfum 150ml",
"sub_category":"Perfumes",
"description":"The House of Chloé unveils a collection of nine exclusive Eau de Parfum fragrances for woman: Atelier des Fleurs.\nMagnolia Alba transcribes the smooth, plump and slightly lemony notes of magnolia blossoms in spring."
}

result=content_runnable.invoke(user_input)
pprint.pprint(result.product_description)

('Discover the essence of spring with CHLOÉ Atelier des Fleurs Magnolia Alba '
 'Eau de Parfum 150ml. This exquisite fragrance transcribes the smooth, plump '
 'notes of magnolia blossoms in bloom, infused with a subtle lemony nuance. '
 'The House of Chloé presents this exclusive collection of nine Eau de Parfum '
 "fragrances for women, each capturing the essence of nature's beauty.")


In [7]:
import requests

# Set up the request to match the curl command
response = requests.post(
    "http://10.2.125.37:1234/v1/chat/completions",
    headers={"Content-Type": "application/json"},
    json={
        "model": "lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
        "messages": [
            {"role": "system", "content": "Always answer in rhymes."},
            {"role": "user", "content": "What are the key benefits of AI in healthcare?"}
        ],
        "temperature": 0.7,
        "max_tokens": 150,
        "stream": False  # Set to True if you want to stream responses, False otherwise
    }
)

# Print the raw response
print("Raw API response:", response.json())


Raw API response: {'id': 'chatcmpl-s4lbqf4srfqeurdmd9w19', 'object': 'chat.completion', 'created': 1725282587, 'model': 'lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "In healthcare, AI's the way,\nTo improve care and seize the day.\nIt brings several benefits, don't you know?\nAccurate diagnoses, with data to show.\n\nPersonalized medicine, it can provide,\nTailored treatments, side effects to divide.\nPredictive analytics, a powerful tool,\nHelping prevent diseases, like a magic rule.\n\nStreamlined workflows, AI takes a bite,\nReducing errors and making things right.\nClinical decision support, always on hand,\nGuiding doctors with expert commands.\n\nData analysis, in real-time too,\nIdentifying trends, and patterns anew.\nThe key benefits of AI in healthcare are clear,\nImproving patient care, without a speck of fear!"}, 'finish_reason': 'stop'}], 'usage': {'prompt_toke